<a href="https://colab.research.google.com/github/SomdeepAcharyya/Recommender-Systems/blob/main/ACTL_Adaptive_Codebook_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ACTL: Adaptive Codebook Transfer Learning for Cross-Domain Recommendation

*MING HE , JIULING ZHANG , AND SHAOZONG ZHANG*

## Headers

In [ ]:
import pandas as pd
import scipy, cmake
import numpy as np
!pip install nosetests 
import cython
!pip install distributions

ERROR: Could not find a version that satisfies the requirement nosetests (from versions: none)
ERROR: No matching distribution found for nosetests
     |████████████████████████████████| 1.5 MB 27.2 MB/s 
  ERROR: Failed building wheel for distributions
  Running setup.py clean for distributions
Failed to build distributions
    Running setup.py install for distributions ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-tc9vx5a0/distributions_f3eb33d48fc346148d53297404c49002/setup.py'"'"'; __file__='"'"'/tmp/pip-install-tc9vx5a0/distributions_f3eb33d48fc346148d53297404c49002/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-ub5

In [ ]:
for i in range(0,100):
  for j in range(0,100):
   get_similarity(item[i], item[j])

arr = []

## Data Reading

In [ ]:
# Amazon books Dataset
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_Books.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  amazon_books = pd.read_csv(infile)
  #amazon_books = amazon_books[['user_id', 'item_id', 'rating']]
  #amazon_books = amazon_books.rename(columns={"user_id":"userId", "item_id": "itemId"})
  #amazon_books.columns
amazon_books.head()

,item_id,user_id,rating,timestamp
0,1713353,A1REUF3A1YCPHM,5,1112140800
1,1713353,A1YRBRK2XM5D5,5,1081036800
2,1713353,A1V8ZR5P78P4ZU,5,1077321600
3,1713353,A2ZB06582NXCIV,5,1475452800
4,1713353,ACPQVNRD3Z09X,5,1469750400


In [ ]:
# Amazon Movies Dataset
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_Movies.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  amazon_movies = pd.read_csv(infile)
  amazon_movies = amazon_movies[['user_id', 'item_id', 'rating']]
  amazon_movies = amazon_movies.rename(columns={"user_id":"userId", "item_id": "itemId"})
  amazon_movies.columns
amazon_movies.head()

,userId,itemId,rating
0,A2VHSG6TZHU1OB,1527665,5
1,A23EJWOW1TLENE,1527665,5
2,A1KM9FNEJ8Q171,1527665,5
3,A38LY2SSHVHRYB,1527665,4
4,AHTYUW2H1276L,1527665,5


In [ ]:
# Amazon Clothes Dataset
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_Clothing.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  amazon_clothes = pd.read_csv(infile)
  amazon_clothes = amazon_clothes[['user_id', 'item_id', 'rating']]
  amazon_clothes = amazon_clothes.rename(columns={"user_id":"userId", "item_id": "itemId"})
  amazon_clothes.columns
amazon_clothes.head()

,userId,itemId,rating
0,A3OT9BYASFGU2X,871167042,4
1,A28GK1G2KDXHRP,871167042,5
2,A3NFXFEKW8OK0E,871167042,5
3,A3I6G5TKBVJEK9,871167042,5
4,A1A7Y1M8AJWNZ8,871167042,5


In [ ]:
# Amazon Video Games Dataset
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_Video_Games.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  amazon_vg = pd.read_csv(infile)
  amazon_vg = amazon_vg[['user_id', 'item_id', 'rating']]
  amazon_vg = amazon_vg.rename(columns={"user_id":"userId", "item_id": "itemId"})
  amazon_vg.columns
amazon_vg.head()

In [ ]:
# MovieLens Dataset

path = r'/content/drive/MyDrive/Per_CD_RS/movielens_ratings.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  movielens = pd.read_csv(infile)
#movielens = movielens[['userId', 'movieId', 'rating']]
#movielens = movielens.rename(columns={"movieId": "itemId"})
movielens.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [ ]:
# BookCrossing Dataset

path = r'/content/drive/MyDrive/Per_CD_RS/BX_Book_Ratings.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  bookcrossing = pd.read_csv(infile)
bookcrossing = bookcrossing[['User-ID', 'ISBN', 'Book-Rating']]
bookcrossing = bookcrossing.rename(columns={"User-ID":"userId", "ISBN":"itemId", "Book-Rating": "rating"})
bookcrossing.columns

Index(['userId', 'itemId', 'rating'], dtype='object')

In [ ]:
# Tripadvisor Dataset

path = r'/content/drive/MyDrive/Per_CD_RS/tripadvisor_reviews.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  tripadvisor = pd.read_csv(infile)
tripadvisor = tripadvisor[['username', 'taObject', 'rating']]
tripadvisor = tripadvisor.rename(columns={"username":"userId", "taObject":"itemId"})
tripadvisor.columns

Index(['userId', 'itemId', 'rating'], dtype='object')

In [ ]:
source_user_ratings = amazon_books[0:10000]
target_user_ratings = amazon_movies[0:10000]

In [ ]:
source_user_ratings= tripadvisor
source_user_ratings

,userId,itemId,rating
0,007solotraveler,"Radisson Blu Royal Viking Hotel, Stockholm",4
1,007solotraveler,Radisson Blu Waterfront Hotel,5
2,007solotraveler,Vasa Museum,5
3,007solotraveler,Hilton Miami Airport,4
4,007solotraveler,Hunter Steakhouse,5
...,...,...,...
32575,Farnborough,Wilma & Albert's Steakhouse,4
32576,Farnborough,Fortuyn Haarlem,5
32577,Farnborough,Beach Club Tien,5
32578,Farnborough,Pension Welkom-in,5


In [ ]:
print(source_user_ratings.shape)
print(target_user_ratings.shape)

(10000, 3)
(10000, 3)


In [ ]:
# creation of target rating matx for matx factorization(MF)
rs = source_user_ratings.pivot_table(index='userId',columns='itemId',values='rating')
print(rs.shape)
# creation of target rating matx for matx factorization(MF)
#rt = target_user_ratings.pivot_table(index='userId',columns='itemId',values='rating')
#print(rt.shape)

(1098, 26300)


In [ ]:
rs

itemId,'Inoteca,'t Cuyperke,'t Doktertje,10 Corso Como Cafe,10 South Rooftop Bar & Grill,10 The Coffee House,10/09 Kitchen,106 Baker St.,107 Dining Room,11@Century Hotel,11th Street Diner,121 Restaurant,1221,127 West Social House,12th Street Beach,137 Pillars House,142 Creperie Contemporaine,15 Church Street Bed & Breakfast - Phillips-Yates-Snowden House,15 Steps,1516 Brewing Company,1539 Restaurant & Bar,158 Main Restaurant & Bakery,15th Street Fisheries,16 Beach Street Bed and Breakfast,16th Street Mall,17-Mile Drive,1801 First Luxury Inn,1826 Restaurant and Lounge,185 Empty White Chairs - Earthquake Memorial,1861 Restaurant,1875 Steakhouse,1877,1886 Crescent Hotel & Spa,1886 Restaurant,1890 Restaurant & Lounge,1897 Beekman House,196 Bishopsgate,1Bite2Go Cafe & Deli,200 Degrees Coffee Shop,200 East On Chestnut,...,miller howe,"minibar, Washington, D.C.",misc eatdrinksleep,moes grill,naked Stables,nhow Rotterdam,"nyma, the New York Manhattan Hotel",olivier bistro,oon dah,pamir,pentahotel Leipzig,pistrina,piyawat thai restaurant,playfood,remm Hibiya,restaurant Ca'l Reiet,restaurant La Stanza,rosa and rocco's,royal barge thai restaurant,ruyi express,san giorgio,seafire snorkel adventures,sumo,sushi Hour,taj,the Farmer and the Fish,the Fenton Fire Hall Kitchen and Taproom,the Lark,the green dragon inn,the sweaty dutchman,theWit - A DoubleTree by Hilton,three tomatoes trattoria,uMhlanga Sands Resort,umi,village taverna,von Stackelberg Hotel Tallinn,wildside grill,wimpy,"with AC, 1BR apt Champ de Mars Near the Eiffel T",wynns restaurant
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
007solotraveler,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0BKI0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10179k79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124_10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12ReasonsWhy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Familyman54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Familyof4Scotland,

In [ ]:
rs = rs.fillna(0)
rs

itemId,'Inoteca,'t Cuyperke,'t Doktertje,10 Corso Como Cafe,10 South Rooftop Bar & Grill,10 The Coffee House,10/09 Kitchen,106 Baker St.,107 Dining Room,11@Century Hotel,11th Street Diner,121 Restaurant,1221,127 West Social House,12th Street Beach,137 Pillars House,142 Creperie Contemporaine,15 Church Street Bed & Breakfast - Phillips-Yates-Snowden House,15 Steps,1516 Brewing Company,1539 Restaurant & Bar,158 Main Restaurant & Bakery,15th Street Fisheries,16 Beach Street Bed and Breakfast,16th Street Mall,17-Mile Drive,1801 First Luxury Inn,1826 Restaurant and Lounge,185 Empty White Chairs - Earthquake Memorial,1861 Restaurant,1875 Steakhouse,1877,1886 Crescent Hotel & Spa,1886 Restaurant,1890 Restaurant & Lounge,1897 Beekman House,196 Bishopsgate,1Bite2Go Cafe & Deli,200 Degrees Coffee Shop,200 East On Chestnut,...,miller howe,"minibar, Washington, D.C.",misc eatdrinksleep,moes grill,naked Stables,nhow Rotterdam,"nyma, the New York Manhattan Hotel",olivier bistro,oon dah,pamir,pentahotel Leipzig,pistrina,piyawat thai restaurant,playfood,remm Hibiya,restaurant Ca'l Reiet,restaurant La Stanza,rosa and rocco's,royal barge thai restaurant,ruyi express,san giorgio,seafire snorkel adventures,sumo,sushi Hour,taj,the Farmer and the Fish,the Fenton Fire Hall Kitchen and Taproom,the Lark,the green dragon inn,the sweaty dutchman,theWit - A DoubleTree by Hilton,three tomatoes trattoria,uMhlanga Sands Resort,umi,village taverna,von Stackelberg Hotel Tallinn,wildside grill,wimpy,"with AC, 1BR apt Champ de Mars Near the Eiffel T",wynns restaurant
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
007solotraveler,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0BKI0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10179k79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
124_10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12ReasonsWhy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Familyman54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Familyof4Scotland,

## Non Negative Matrix Factorization

In [ ]:
# creation of source rating matx for matx factorization(MF)
rs = source_user_ratings.pivot(index='itemId',columns='userId',values='rating')
rs.fillna(0, inplace=True)
rs.shape

(10325, 668)

In [ ]:
# creation of target rating matx for matx factorization(MF)
rt = target_user_ratings.pivot_table(index='itemId',columns='userId',values='rating')
rt.fillna(0, inplace=True)
rt.shape

(9340, 941)

In [ ]:
import numpy as np
X_s = np.array(rs)
X_t = np.array(rt)

In [ ]:
from sklearn.decomposition import NMF
model_s = NMF(n_components=2, init='random', random_state=0)
W_s= model_s.fit_transform(X_s)
H_s = model_s.components_

In [ ]:
model_t = NMF(n_components=2, init='random', random_state=0)
W_t= model_t.fit_transform(X_t)
H_t = model_t.components_

In [ ]:
us = W_s
vs = H_s
ut = W_t
vt = H_t

In [ ]:
H_s.shape

(2, 668)

## Non-Negative Matrix Tri Factorization 

In [ ]:
# orthogonal non negative matrix tri factorizations for clustering by Ding, Li, Peng, Park; Association for computing machinery, ACM

In [ ]:
import numpy, random, time

max_iterations = 200 # safeguard - if it takes more than this many iterations, stop

# Cluster the rows of the dataset X, with missing values indicated by M.
class KMeansAlgo:
    def __init__(self,X,M,K,resolve_empty='singleton'):
        self.X = numpy.copy(X)
        self.M = numpy.copy(M)
        self.K = K
        self.resolve_empty = resolve_empty
        
        assert len(self.X.shape) == 2, "Input matrix X is not a two-dimensional array, but instead %s-dimensional." % len(self.X.shape)
        assert self.X.shape == self.M.shape, "Input matrix X is not of the same size as the indicator matrix M: %s and %s respectively." % (self.X.shape,self.M.shape)
        assert self.K > 0, "K should be greater than 0."
        
        (self.no_points,self.no_coordinates) = self.X.shape
        self.no_unique_points = len(set([tuple(l) for l in self.X.tolist()]))    
        
        if self.no_points < self.K: print("Want %s clusters but only have %s datapoints!" % (self.K,self.no_points))
        if self.no_unique_points < self.K: print("Want %s clusters but only have %s unique datapoints!" % (self.K,self.no_unique_points))
        
        # Compute lists of which indices are known (from M) for each row/column
        self.omega_rows = [[j for j in range(0,self.no_coordinates) if self.M[i,j]] for i in range(0,self.no_points)]        
        self.omega_columns = [[i for i in range(0,self.no_points) if self.M[i,j]] for j in range(0,self.no_coordinates)]
                    
        # Assert none of the rows are entirely unknown values
        for i,omega_row in enumerate(self.omega_rows):
            assert len(omega_row) != 0, "Fully unobserved row in X, row %s." % i
            
        # Columns can be entirely unknown - they just don't influence the clustering - but we need to remove them
        columns_to_remove = [j for j,omega_column in enumerate(self.omega_columns) if len(omega_column) == 0]
        if len(columns_to_remove) > 0:
            print("WARNING: removed columns %s for K-means clustering as they have no observed datapoints." % columns_to_remove)
            self.X = numpy.delete(self.X,columns_to_remove,axis=1)
            self.M = numpy.delete(self.M,columns_to_remove,axis=1)
            self.no_coordinates -= len(columns_to_remove)
            self.omega_rows = [[j for j in range(0,self.no_coordinates) if self.M[i,j]] for i in range(0,self.no_points)]        
            self.omega_columns = [[i for i in range(0,self.no_points) if self.M[i,j]] for j in range(0,self.no_coordinates)]
            
        # Initialise the distances from data points to the assigned cluster centroids to zeros
        self.distances = numpy.zeros(self.no_points)
    
    
    """ Initialise the cluster centroids randomly """
    def initialise(self,seed=None):
        if seed is not None:
            random.seed(seed)
        
        # Compute the mins and maxes of the columns - i.e. the min and max of each dimension
        self.mins = [min([self.X[i,j] for i in self.omega_columns[j]]) for j in range(0,self.no_coordinates)]
        self.maxs = [max([self.X[i,j] for i in self.omega_columns[j]]) for j in range(0,self.no_coordinates)]
        
        # Randomly initialise the cluster centroids
        self.centroids = [self.random_cluster_centroid() for k in xrange(0,self.K)]
        self.cluster_assignments = numpy.array([-1 for d in xrange(0,self.no_points)])
        self.mask_centroids = numpy.ones((self.K,self.no_coordinates))


    # Randomly place a new cluster centroids, picking uniformly between the min and max of each coordinate
    def random_cluster_centroid(self):
        centroid = []
        for coordinate in xrange(0,self.no_coordinates):
            value = random.uniform(self.mins[coordinate],self.maxs[coordinate])
            centroid.append(value)     
        return centroid    
    
            
    """ Perform the clustering, until there is no change """
    def cluster(self):
        iteration = 1
        change = True
        while change:
            print("Iteration: %s." % iteration)
            iteration += 1
            change = self.assignment()
            self.update()
            
            if iteration >= max_iterations:
                print("WARNING: did not converge, stopped after %s iterations." % max_iterations)
                break
            
        # At the end, we create a binary matrix indicating which points were assigned to which cluster
        self.create_matrix()


    """ Assign each data point to the closest cluster, and return whether any reassignments were made """
    def assignment(self):
        self.data_point_assignments = [[] for k in xrange(0,self.K)]
        
        change = False
        for d,(data_point,mask) in enumerate(zip(self.X,self.M)):
            old_c = self.cluster_assignments[d]
            new_c = self.closest_cluster(data_point,d,mask)
            
            self.cluster_assignments[d] = new_c
            self.data_point_assignments[new_c].append(d)
            
            change = (change or old_c != new_c)
        return change
    
    
    # Compute the MSE to each of the clusters, and return the index of the closest cluster.
    # If two clusters are equally far, we return the cluster with the lowest index.
    def closest_cluster(self,data_point,index_data_point,mask_d):
        closest_index = None
        closest_MSE = None
        for c,(centroid,mask_c) in enumerate(zip(self.centroids,self.mask_centroids)):
            MSE = self.compute_MSE(data_point,centroid,mask_d,mask_c)
            if (closest_MSE is None) or (MSE is not None and MSE < closest_MSE): #either no closest centroid yet, or MSE is defined and less
                closest_MSE = MSE
                closest_index = c
        self.distances[index_data_point] = closest_MSE
        return closest_index
    
    
    # Compute the Euclidean distance between the data point and the cluster centroid.
    # If they have no known values in common, we return None (=infinite distance).
    def compute_MSE(self,x1,x2,mask1,mask2):
        mask1_mask2 = numpy.multiply(mask1,mask2)
        len_overlap = mask1_mask2.sum()
        return None if len_overlap == 0 else numpy.multiply(mask1_mask2,numpy.power(numpy.subtract(x1,x2),2)).sum() / float(len_overlap)
            
        
    """ Update the centroids to the mean of the points assigned to it. 
        If for a coordinate there are no known values, we set this cluster's mask to 0 there.
        If a cluster has no points assigned to it at all, we randomly re-initialise it. """
    def update(self):
        for c in xrange(0,self.K): 
            self.update_cluster(c)
    
    # Update for one specific cluster
    def update_cluster(self,c):
        known_coordinate_values = self.find_known_coordinate_values(c)
        
        if known_coordinate_values is None:
            # Reassign a datapoint to this cluster, as long as there are enough 
            # unique datapoints. Either furthest away (singleton) or random.
            if self.no_unique_points >= self.K:
                if self.resolve_empty == 'singleton':
                    # Find the point currently furthest away from its centroid                    
                    index_furthest_away = self.find_point_furthest_away()
                    old_cluster = self.cluster_assignments[index_furthest_away]
                    
                    # Add point to new cluster
                    self.centroids[c] = self.X[index_furthest_away]
                    self.mask_centroids[c] = self.M[index_furthest_away]
                    self.distances[index_furthest_away] = 0.0
                    self.cluster_assignments[index_furthest_away] = c
                    self.data_point_assignments[c] = [index_furthest_away]
                    
                    # Remove from old cluster and update
                    self.data_point_assignments[old_cluster].remove(index_furthest_away)
                    #print c,old_cluster
                    self.update_cluster(old_cluster)
                else:
                    # Randomly re-initialise this point
                    self.centroids[c] = self.random_cluster_centroid()
                    self.mask_centroids[c] = numpy.ones(self.no_coordinates)
        else:
            # For each coordinate set the centroid to the average, or to 0 if no values are observed
            for coordinate,coordinate_values in enumerate(known_coordinate_values):
                if len(coordinate_values) == 0:
                    new_coordinate = 0              
                    new_mask = 0
                else:
                    new_coordinate = sum(coordinate_values) / float(len(coordinate_values))
                    new_mask = 1
                
                self.centroids[c][coordinate] = new_coordinate
                self.mask_centroids[c][coordinate] = new_mask
    
    
    # For a given centroid c, construct a list of lists, each list consisting of
    # all known coordinate values of data points assigned to the centroid.
    # If no points are assigned to a cluster, return None.
    def find_known_coordinate_values(self,c):
        assigned_data_indexes = self.data_point_assignments[c]
        data_points = numpy.array([self.X[d] for d in assigned_data_indexes])
        masks = numpy.array([self.M[d] for d in assigned_data_indexes])
        
        if len(assigned_data_indexes) == 0:
            lists_known_coordinate_values = None
        else: 
            lists_known_coordinate_values = [
                [v for d,v in enumerate(data_points.T[coordinate]) if masks[d][coordinate]]
                for coordinate in xrange(0,self.no_coordinates)
            ]
            
        return lists_known_coordinate_values
        
    
    # Find data point furthest away from its current cluster centroid
    def find_point_furthest_away(self):
        data_point_index = self.distances.argmax()
        return data_point_index
        
    
    # Create a binary matrix indicating the clustering (so size [no_points x K])
    def create_matrix(self):
        self.clustering_results = numpy.zeros((self.no_points,self.K))
        for d in range(0,self.no_points):
            c = self.cluster_assignments[d]
            self.clustering_results[d][c] = 1
            
        print([sum(column) for column in self.clustering_results.T])

In [ ]:
#from kmeans.kmeans import KMeans
KMeans = KMeansAlgo(rs, rs, 50)
#from distributions.exponential import exponential_draw

import numpy,itertools,math,time

class NMTF:
    def __init__(self,R,M,K,L):
        self.R = numpy.array(R,dtype=float)
        self.M = numpy.array(M,dtype=float)
        self.K = K            
        self.L = L    
        
        self.metrics = ['MSE','R^2','Rp']
                
        assert len(self.R.shape) == 2, "Input matrix R is not a two-dimensional array, " \
            "but instead %s-dimensional." % len(self.R.shape)
        assert self.R.shape == self.M.shape, "Input matrix R is not of the same size as " \
            "the indicator matrix M: %s and %s respectively." % (self.R.shape,self.M.shape)
        
        (self.I,self.J) = self.R.shape
        
        self.check_empty_rows_columns() 
        
        # For computing the I-div it is better if unknown values are 1's, not 0's
        self.R_excl_unknown = numpy.empty((self.I,self.J))
        for i,j in itertools.product(range(0,self.I),range(0,self.J)):
            self.R_excl_unknown[i,j] = self.R[i,j] if self.M[i,j] else 1.
                 
                 
    # Raise an exception if an entire row or column is empty
    def check_empty_rows_columns(self):
        sums_columns = self.M.sum(axis=0)
        sums_rows = self.M.sum(axis=1)
                    
        # Assert none of the rows or columns are entirely unknown values
        for i,c in enumerate(sums_rows):
            assert c != 0, "Fully unobserved row in R, row %s." % i
        for j,c in enumerate(sums_columns):
            assert c != 0, "Fully unobserved column in R, column %s." % j
                 

    """ Initialise F, S and G """    
    def initialise(self,init_S='exponential',init_FG='exponential',expo_prior=1.):
        assert init_S in ['ones','random','exponential'], "Unrecognised init option for S: %s." % init_S
        assert init_FG in ['ones','random','exponential','kmeans'], "Unrecognised init option for F,G: %s." % init_FG
        
        if init_S == 'ones':
            self.S = numpy.ones((self.K,self.L))
        elif init_S == 'random':
            self.S = numpy.random.rand(self.K,self.L)
        elif init_S == 'exponential':
            self.S = numpy.empty((self.K,self.L))
            for k,l in itertools.product(xrange(0,self.K),xrange(0,self.L)):        
                #self.S[k,l] = exponential_draw(expo_prior)
                self.S[k,l] = numpy.random.exponential(scale = expo_prior)
        
        if init_FG == 'ones':
            self.F = numpy.ones((self.I,self.K))
            self.G = numpy.ones((self.J,self.L))
        elif init_FG == 'random':
            self.F = numpy.random.rand(self.I,self.K)
            self.G = numpy.random.rand(self.J,self.L)
        elif init_FG == 'exponential':
            self.F = numpy.empty((self.I,self.K))
            self.G = numpy.empty((self.J,self.L))
            for i,k in itertools.product(xrange(0,self.I),xrange(0,self.K)):        
                #self.F[i,k] = exponential_draw(expo_prior)
                self.F[i,k] = numpy.random.exponential(scale = expo_prior)
            for j,l in itertools.product(xrange(0,self.J),xrange(0,self.L)):
                #self.G[j,l] = exponential_draw(expo_prior)
                self.G[j,l] = numpy.random.exponential(scale = expo_prior)
        elif init_FG == 'kmeans':
            print("Initialising F using KMeans.")
            kmeans_F = KMeans(self.R,self.M,self.K)
            kmeans_F.initialise()
            kmeans_F.cluster()
            self.F = kmeans_F.clustering_results + 0.2            
            
            print("Initialising G using KMeans.")
            kmeans_G = KMeans(self.R.T,self.M.T,self.L)   
            kmeans_G.initialise()
            kmeans_G.cluster()
            self.G = kmeans_G.clustering_results + 0.2
        
        
    """ Update F, S, G for a number of iterations, printing the performances each iteration. """
    def run(self,iterations):
        assert hasattr(self,'F') and hasattr(self,'S') and hasattr(self,'G'), \
            "F, S and G have not been initialised - please run NMTF.initialise() first."        
        
        self.all_times = [] # to plot performance against time
        self.all_performances = {} # for plotting convergence of metrics
        for metric in self.metrics:
            self.all_performances[metric] = []
            
        time_start = time.time()
        for it in range(1,iterations+1):
            # Doing S first gives more interpretable results (F,G ~= [0,1] rather than [0,20])
            for k,l in itertools.product(range(0,self.K),xrange(0,self.L)):
                self.update_S(k,l)
                    
            for k in range(0,self.K):
                self.update_F(k)
                
            for l in range(0,self.L):
                self.update_G(l)
               
            self.give_update(it)
            
            time_iteration = time.time()
            self.all_times.append(time_iteration-time_start)  
        
        
    """ Method for doing both initialise() and run() """
    def train(self,iterations,init_S='random',init_FG='random',expo_prior=1.):
        self.initialise(init_S=init_S,init_FG=init_FG,expo_prior=expo_prior) 
        self.run(iterations=iterations)
        
                
    """ Updates for F, G, S. """                
    # Compute the dot product of three matrices
    def triple_dot(self,M1,M2,M3):
        return numpy.dot(M1,numpy.dot(M2,M3))
        
    def update_F(self,k):
        R_pred = self.triple_dot(self.F,self.S,self.G.T)
        SG = numpy.dot(self.S[k],self.G.T)
        numerator = (self.M * self.R / R_pred * SG).sum(axis=1)
        denominator = (self.M * SG).sum(axis=1)
        self.F[:,k] = self.F[:,k] * numerator / denominator
        
    def update_G(self,l):
        R_pred = self.triple_dot(self.F,self.S,self.G.T)
        FS = numpy.dot(self.F,self.S[:,l])
        numerator = ((self.M * self.R / R_pred).T * FS).T.sum(axis=0)
        denominator = (self.M.T * FS).T.sum(axis=0)
        self.G[:,l] = self.G[:,l] * numerator / denominator
        
    def update_S(self,k,l):
        R_pred = self.triple_dot(self.F,self.S,self.G.T)
        F_times_G = self.M * numpy.outer(self.F[:,k], self.G[:,l])   
        numerator = (self.R * F_times_G / R_pred).sum()
        denominator = F_times_G.sum()
        self.S[k,l] = self.S[k,l] * numerator / denominator
           
           
    ''' Functions for computing MSE, R^2 (coefficient of determination), Rp (Pearson correlation) '''
    def predict(self,M_pred):
        R_pred = self.triple_dot(self.F,self.S,self.G.T)
        MSE = self.compute_MSE(M_pred,self.R,R_pred)
        R2 = self.compute_R2(M_pred,self.R,R_pred)    
        Rp = self.compute_Rp(M_pred,self.R,R_pred)        
        return {'MSE':MSE,'R^2':R2,'Rp':Rp}        
        
    def compute_MSE(self,M,R,R_pred):
        return (M * (R-R_pred)**2).sum() / float(M.sum())
        
    def compute_R2(self,M,R,R_pred):
        mean = (M*R).sum() / float(M.sum())
        SS_total = float((M*(R-mean)**2).sum())
        SS_res = float((M*(R-R_pred)**2).sum())
        return 1. - SS_res / SS_total if SS_total != 0. else numpy.inf
        
    def compute_Rp(self,M,R,R_pred):
        mean_real = (M*R).sum() / float(M.sum())
        mean_pred = (M*R_pred).sum() / float(M.sum())
        covariance = (M*(R-mean_real)*(R_pred-mean_pred)).sum()
        variance_real = (M*(R-mean_real)**2).sum()
        variance_pred = (M*(R_pred-mean_pred)**2).sum()
        return covariance / float(math.sqrt(variance_real)*math.sqrt(variance_pred))   
        
    def compute_I_div(self):    
        R_pred = self.triple_dot(self.F,self.S,self.G.T)
        return (self.M * ( self.R_excl_unknown * numpy.log( self.R_excl_unknown / R_pred ) - self.R_excl_unknown + R_pred ) ).sum()        
        
        
    """ Give updates and store performances """
    def give_update(self,iteration):    
        perf = self.predict(self.M)
        i_div = self.compute_I_div()
        
        for metric in self.metrics:
            self.all_performances[metric].append(perf[metric])
               
        print("Iteration %s. I-divergence: %s. MSE: %s. R^2: %s. Rp: %s." % (iteration,i_div,perf['MSE'],perf['R^2'],perf['Rp']))

In [ ]:
# Replacing nan values with 0
rs_n = rs.replace(np.nan, 0)
rt_n = rt.replace(np.nan, 0)

In [ ]:
# create mask array for rs and rt where unobserved as 0 and observed as 1. any value with non zero rating is observed value
rs_m = rs_n > 0
rt_m = rt_n > 0

In [ ]:
rs_m = rs_m.replace(True, 1)
rs_m = rs_m.replace(False, 0)
rt_m = rt_m.replace(True, 1)
rt_m = rt_m.replace(False, 0)

## SVD Matrix Factorization

In [ ]:
# matrix factorization(MF) via singular value decomposition(SVD) ad bm25_weight

!pip install implicit
import scipy.sparse.linalg
from implicit.nearest_neighbours import bm25_weight
vt, x1, ut = scipy.sparse.linalg.svds(bm25_weight(rt), 50)
vs, x2, us = scipy.sparse.linalg.svds(bm25_weight(rs), 50)

     |████████████████████████████████| 1.1MB 13.3MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3406421 sha256=2b6b192805f75e20d75b2c6f00f497ba98d5dc5bcbda0cc484b021a57a6d98b1
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [ ]:
vt.shape
a = np.transpose(vt)
a.shape
vt.shape

(9340, 50)

In [ ]:
rt.shape

(9340, 941)

In [ ]:
a = np.matmul(np.transpose(vt), rt)

ValueError: ignored

## Non-probabilistic non-negative matrix tri-factorisation

In [ ]:
# Non-probabilistic non-negative matrix tri-factorisation, as presented in
#"Probabilistic Matrix Tri-Factorisation" (Yoo and Choi, 2009).

In [ ]:
# Replacing nan values with 0
rs_n = rs.replace(np.nan, 0)
rt_n = rt.replace(np.nan, 0)

In [ ]:
# create mask array for rs and rt where unobserved as 0 and observed as 1. any value with non zero rating is observed value
rs_m = rs_n > 0
rt_m = rt_n > 0

In [ ]:
rs_m

itemId,000047715X,000105001X,000171287X,000211495X,000216132X,1046519,1047566,1052292,1061240,13765,1381733,1384198,1472933,1473123,1477889,1711296,1712764,1712799,1712845,1713205,1713353,1714384,1714538,1714716,1716069,1720279,1720392,1839225,1839233,1840266,1840851,1841572,1844423,1857290,1921517,1932349,1937987,1945424,1950061,1954563,1983008,2000318,2005018,2005263,2005379,2005395,2006448,2006995,2007649,2007908,2007916,2008343,2008572,2008599,2051850,2103826,2111306,2112256,2117010,2117088,2152525,2153211,2153971,2160595,2163578,230022,913154
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A0334855HN6E38CXWXZR,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
A0520969JGW2CZRMEZMM,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
A0790722OCX87RKL2J3T,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
A100ATCZ8R81KG,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
A101M64MW6EH3O,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZVK41090G5LI,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
AZVVRNCE2F0ZY,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
AZWV6VDHSTION,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Tr

In [ ]:
rs_m = rs_m.replace(True, 1)
rs_m = rs_m.replace(False, 0)
rt_m = rt_m.replace(True, 1)
rt_m = rt_m.replace(False, 0)
rs_m

itemId,000047715X,000105001X,000171287X,000211495X,000216132X,1046519,1047566,1052292,1061240,13765,1381733,1384198,1472933,1473123,1477889,1711296,1712764,1712799,1712845,1713205,1713353,1714384,1714538,1714716,1716069,1720279,1720392,1839225,1839233,1840266,1840851,1841572,1844423,1857290,1921517,1932349,1937987,1945424,1950061,1954563,1983008,2000318,2005018,2005263,2005379,2005395,2006448,2006995,2007649,2007908,2007916,2008343,2008572,2008599,2051850,2103826,2111306,2112256,2117010,2117088,2152525,2153211,2153971,2160595,2163578,230022,913154
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A0334855HN6E38CXWXZR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0520969JGW2CZRMEZMM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0790722OCX87RKL2J3T,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A100ATCZ8R81KG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A101M64MW6EH3O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZVK41090G5LI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZVVRNCE2F0ZY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZWV6VDHSTION,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import numpy, random, time

max_iterations = 200 # safeguard - if it takes more than this many iterations, stop

# Cluster the rows of the dataset X, with missing values indicated by M.
class KMeans:
    def __init__(self,X,M,K,resolve_empty='singleton'):
        self.X = numpy.copy(X)
        self.M = numpy.copy(M)
        self.K = K
        self.resolve_empty = resolve_empty
        
        assert len(self.X.shape) == 2, "Input matrix X is not a two-dimensional array, but instead %s-dimensional." % len(self.X.shape)
        assert self.X.shape == self.M.shape, "Input matrix X is not of the same size as the indicator matrix M: %s and %s respectively." % (self.X.shape,self.M.shape)
        assert self.K > 0, "K should be greater than 0."
        
        (self.no_points,self.no_coordinates) = self.X.shape
        self.no_unique_points = len(set([tuple(l) for l in self.X.tolist()]))    
        
        if self.no_points < self.K: print("Want %s clusters but only have %s datapoints!" % (self.K,self.no_points))
        if self.no_unique_points < self.K: print("Want %s clusters but only have %s unique datapoints!" % (self.K,self.no_unique_points))
        
        # Compute lists of which indices are known (from M) for each row/column
        self.omega_rows = [[j for j in range(0,self.no_coordinates) if self.M[i,j]] for i in range(0,self.no_points)]        
        self.omega_columns = [[i for i in range(0,self.no_points) if self.M[i,j]] for j in range(0,self.no_coordinates)]
                    
        # Assert none of the rows are entirely unknown values
        for i,omega_row in enumerate(self.omega_rows):
            assert len(omega_row) != 0, "Fully unobserved row in X, row %s." % i
            
        # Columns can be entirely unknown - they just don't influence the clustering - but we need to remove them
        columns_to_remove = [j for j,omega_column in enumerate(self.omega_columns) if len(omega_column) == 0]
        if len(columns_to_remove) > 0:
            print("WARNING: removed columns %s for K-means clustering as they have no observed datapoints." % columns_to_remove)
            self.X = numpy.delete(self.X,columns_to_remove,axis=1)
            self.M = numpy.delete(self.M,columns_to_remove,axis=1)
            self.no_coordinates -= len(columns_to_remove)
            self.omega_rows = [[j for j in range(0,self.no_coordinates) if self.M[i,j]] for i in range(0,self.no_points)]        
            self.omega_columns = [[i for i in range(0,self.no_points) if self.M[i,j]] for j in range(0,self.no_coordinates)]
            
        # Initialise the distances from data points to the assigned cluster centroids to zeros
        self.distances = numpy.zeros(self.no_points)
    
    
    """ Initialise the cluster centroids randomly """
    def initialise(self,seed=None):
        if seed is not None:
            random.seed(seed)
        
        # Compute the mins and maxes of the columns - i.e. the min and max of each dimension
        self.mins = [min([self.X[i,j] for i in self.omega_columns[j]]) for j in range(0,self.no_coordinates)]
        self.maxs = [max([self.X[i,j] for i in self.omega_columns[j]]) for j in range(0,self.no_coordinates)]
        
        # Randomly initialise the cluster centroids
        self.centroids = [self.random_cluster_centroid() for k in range(0,self.K)]
        self.cluster_assignments = numpy.array([-1 for d in range(0,self.no_points)])
        self.mask_centroids = numpy.ones((self.K,self.no_coordinates))


    # Randomly place a new cluster centroids, picking uniformly between the min and max of each coordinate
    def random_cluster_centroid(self):
        centroid = []
        for coordinate in range(0,self.no_coordinates):
            value = random.uniform(self.mins[coordinate],self.maxs[coordinate])
            centroid.append(value)     
        return centroid    
    
            
    """ Perform the clustering, until there is no change """
    def cluster(self):
        iteration = 1
        change = True
        while change:
            print("Iteration: %s." % iteration)
            iteration += 1
            change = self.assignment()
            self.update()
            
            if iteration >= max_iterations:
                print("WARNING: did not converge, stopped after %s iterations." % max_iterations)
                break
            
        # At the end, we create a binary matrix indicating which points were assigned to which cluster
        self.create_matrix()


    """ Assign each data point to the closest cluster, and return whether any reassignments were made """
    def assignment(self):
        self.data_point_assignments = [[] for k in range(0,self.K)]
        
        change = False
        for d,(data_point,mask) in enumerate(zip(self.X,self.M)):
            old_c = self.cluster_assignments[d]
            new_c = self.closest_cluster(data_point,d,mask)
            
            self.cluster_assignments[d] = new_c
            self.data_point_assignments[new_c].append(d)
            
            change = (change or old_c != new_c)
        return change
    
    
    # Compute the MSE to each of the clusters, and return the index of the closest cluster.
    # If two clusters are equally far, we return the cluster with the lowest index.
    def closest_cluster(self,data_point,index_data_point,mask_d):
        closest_index = None
        closest_MSE = None
        for c,(centroid,mask_c) in enumerate(zip(self.centroids,self.mask_centroids)):
            MSE = self.compute_MSE(data_point,centroid,mask_d,mask_c)
            if (closest_MSE is None) or (MSE is not None and MSE < closest_MSE): #either no closest centroid yet, or MSE is defined and less
                closest_MSE = MSE
                closest_index = c
        self.distances[index_data_point] = closest_MSE
        return closest_index
    
    
    # Compute the Euclidean distance between the data point and the cluster centroid.
    # If they have no known values in common, we return None (=infinite distance).
    def compute_MSE(self,x1,x2,mask1,mask2):
        mask1_mask2 = numpy.multiply(mask1,mask2)
        len_overlap = mask1_mask2.sum()
        return None if len_overlap == 0 else numpy.multiply(mask1_mask2,numpy.power(numpy.subtract(x1,x2),2)).sum() / float(len_overlap)
            
        
    """ Update the centroids to the mean of the points assigned to it. 
        If for a coordinate there are no known values, we set this cluster's mask to 0 there.
        If a cluster has no points assigned to it at all, we randomly re-initialise it. """
    def update(self):
        for c in range(0,self.K): 
            self.update_cluster(c)
    
    # Update for one specific cluster
    def update_cluster(self,c):
        known_coordinate_values = self.find_known_coordinate_values(c)
        
        if known_coordinate_values is None:
            # Reassign a datapoint to this cluster, as long as there are enough 
            # unique datapoints. Either furthest away (singleton) or random.
            if self.no_unique_points >= self.K:
                if self.resolve_empty == 'singleton':
                    # Find the point currently furthest away from its centroid                    
                    index_furthest_away = self.find_point_furthest_away()
                    old_cluster = self.cluster_assignments[index_furthest_away]
                    
                    # Add point to new cluster
                    self.centroids[c] = self.X[index_furthest_away]
                    self.mask_centroids[c] = self.M[index_furthest_away]
                    self.distances[index_furthest_away] = 0.0
                    self.cluster_assignments[index_furthest_away] = c
                    self.data_point_assignments[c] = [index_furthest_away]
                    
                    # Remove from old cluster and update
                    self.data_point_assignments[old_cluster].remove(index_furthest_away)
                    #print c,old_cluster
                    self.update_cluster(old_cluster)
                else:
                    # Randomly re-initialise this point
                    self.centroids[c] = self.random_cluster_centroid()
                    self.mask_centroids[c] = numpy.ones(self.no_coordinates)
        else:
            # For each coordinate set the centroid to the average, or to 0 if no values are observed
            for coordinate,coordinate_values in enumerate(known_coordinate_values):
                if len(coordinate_values) == 0:
                    new_coordinate = 0              
                    new_mask = 0
                else:
                    new_coordinate = sum(coordinate_values) / float(len(coordinate_values))
                    new_mask = 1
                
                self.centroids[c][coordinate] = new_coordinate
                self.mask_centroids[c][coordinate] = new_mask
    
    
    # For a given centroid c, construct a list of lists, each list consisting of
    # all known coordinate values of data points assigned to the centroid.
    # If no points are assigned to a cluster, return None.
    def find_known_coordinate_values(self,c):
        assigned_data_indexes = self.data_point_assignments[c]
        data_points = numpy.array([self.X[d] for d in assigned_data_indexes])
        masks = numpy.array([self.M[d] for d in assigned_data_indexes])
        
        if len(assigned_data_indexes) == 0:
            lists_known_coordinate_values = None
        else: 
            lists_known_coordinate_values = [
                [v for d,v in enumerate(data_points.T[coordinate]) if masks[d][coordinate]]
                for coordinate in range(0,self.no_coordinates)
            ]
            
        return lists_known_coordinate_values
        
    
    # Find data point furthest away from its current cluster centroid
    def find_point_furthest_away(self):
        data_point_index = self.distances.argmax()
        return data_point_index
        
    
    # Create a binary matrix indicating the clustering (so size [no_points x K])
    def create_matrix(self):
        self.clustering_results = numpy.zeros((self.no_points,self.K))
        for d in range(0,self.no_points):
            c = self.cluster_assignments[d]
            self.clustering_results[d][c] = 1
            
        print([sum(column) for column in self.clustering_results.T])

In [ ]:
"""
Non-probabilistic non-negative matrix tri-factorisation, as presented in
"Probabilistic Matrix Tri-Factorisation" (Yoo and Choi, 2009).
We change the notation to match ours: R = FSG.T instead of V = USV.T.
The updates are then:
- Uik <- Uik * (sum_j Vjk * Rij / (Ui dot Vj)) / (sum_j Vjk)
- Vjk <- Vjk * (sum_i Uik * Rij / (Ui dot Vj)) / (sum_i Uik)
Or more efficiently using matrix operations:
- Uik <- Uik * (Mi dot [V.k * Ri / (Ui dot V.T)]) / (Mi dot V.k)
- Vjk <- Vjk * (M.j dot [U.k * R.j / (U dot Vj)]) / (M.j dot U.k)
And realising that elements in each column in U and V are independent:
- U.k <- U.k * sum(M * [V.k * (R / (U dot V.T))], axis=1) / sum(M dot V.k, axis=1)
- V.k <- V.k * sum(M * [U.k * (R / (U dot V.T))], axis=0) / sum(M dot U.k, axis=0)
We expect the following arguments:
- R, the matrix
- M, the mask matrix indicating observed values (1) and unobserved ones (0)
- K, the number of row latent factors
- L, the number of column latent factors
    
Initialisation can be done by running the initialise(init,tauUV) function. We initialise as follows:
- init_S = 'ones'          -> S[i,k] = 1
         = 'random'        -> S[i,k] ~ U(0,1)
         = 'exponential'   -> S[i,k] ~ Exp(expo_prior)
- init_FG = 'ones'          -> F[i,k] = G[j,k] = 1
          = 'random'        -> F[i,k] ~ U(0,1), G[j,l] ~ G(0,1), 
          = 'exponential'   -> F[i,k] ~ Exp(expo_prior), G[j,l] ~ Exp(expo_prior) 
          = 'kmeans'        -> F = KMeans(R,rows)+0.2, G = KMeans(R,columns)+0.2
  where expo_prior is an additional parameter (default 1)
"""

#from kmeans.kmeans import KMeans
#from sklearn.cluster import KMeans
#from distributions.exponential import exponential_draw

import numpy,itertools,math,time

class NMTF:
    def __init__(self,R,M,K,L):
        self.R = numpy.array(R,dtype=float)
        self.M = numpy.array(M,dtype=float)
        self.K = K            
        self.L = L    
        
        self.metrics = ['MSE','R^2','Rp']
                
        assert len(self.R.shape) == 2, "Input matrix R is not a two-dimensional array, " \
            "but instead %s-dimensional." % len(self.R.shape)
        assert self.R.shape == self.M.shape, "Input matrix R is not of the same size as " \
            "the indicator matrix M: %s and %s respectively." % (self.R.shape,self.M.shape)
        
        (self.I,self.J) = self.R.shape
        
        self.check_empty_rows_columns() 
        
        # For computing the I-div it is better if unknown values are 1's, not 0's
        self.R_excl_unknown = numpy.empty((self.I,self.J))
        for i,j in itertools.product(range(0,self.I),range(0,self.J)):
            self.R_excl_unknown[i,j] = self.R[i,j] if self.M[i,j] else 1.
                 
                 
    # Raise an exception if an entire row or column is empty
    def check_empty_rows_columns(self):
        sums_columns = self.M.sum(axis=0)
        sums_rows = self.M.sum(axis=1)
                    
        # Assert none of the rows or columns are entirely unknown values
        for i,c in enumerate(sums_rows):
            assert c != 0, "Fully unobserved row in R, row %s." % i
        for j,c in enumerate(sums_columns):
            assert c != 0, "Fully unobserved column in R, column %s." % j
                 

    """ Initialise F, S and G """    
    def initialise(self,init_S='random',init_FG='random',expo_prior=1.):
        assert init_S in ['ones','random','exponential'], "Unrecognised init option for S: %s." % init_S
        assert init_FG in ['ones','random','exponential','kmeans'], "Unrecognised init option for F,G: %s." % init_FG
        
        if init_S == 'ones':
            self.S = numpy.ones((self.K,self.L))
        elif init_S == 'random':
            self.S = numpy.random.rand(self.K,self.L)
        elif init_S == 'exponential':
            self.S = numpy.empty((self.K,self.L))
            for k,l in itertools.product(range(0,self.K),range(0,self.L)):        
                #self.S[k,l] = exponential_draw(expo_prior)
                self.S[k,l] = numpy.random.exponential(scale = expo_prior)
        
        if init_FG == 'ones':
            self.F = numpy.ones((self.I,self.K))
            self.G = numpy.ones((self.J,self.L))
        elif init_FG == 'random':
            self.F = numpy.random.rand(self.I,self.K)
            self.G = numpy.random.rand(self.J,self.L)
        elif init_FG == 'exponential':
            self.F = numpy.empty((self.I,self.K))
            self.G = numpy.empty((self.J,self.L))
            for i,k in itertools.product(range(0,self.I),range(0,self.K)):        
                #self.F[i,k] = exponential_draw(expo_prior)
                self.F[i,k] = numpy.random.exponential(scale = expo_prior)
            for j,l in itertools.product(range(0,self.J),range(0,self.L)):
                #self.G[j,l] = exponential_draw(expo_prior)
                self.G[j,l] = numpy.random.exponential(scale = expo_prior)
        elif init_FG == 'kmeans':
            print("Initialising F using KMeans.")
            kmeans_F = KMeans(self.R,self.M,self.K)
            #KMeans(n_clusters=self.K, random_state=0).fit(self.R)
            kmeans_F.initialise()
            kmeans_F.cluster()
            self.F = kmeans_F.clustering_results + 0.2            
            
            print("Initialising G using KMeans.")
            kmeans_G = KMeans(self.R.T,self.M.T,self.L)
            #KMeans(n_clusters=self.L, random_state=0).fit(self.R.T)   
            kmeans_G.initialise()
            kmeans_G.cluster()
            self.G = kmeans_G.clustering_results + 0.2
        
        
    """ Update F, S, G for a number of iterations, printing the performances each iteration. """
    def run(self,iterations):
        assert hasattr(self,'F') and hasattr(self,'S') and hasattr(self,'G'), \
            "F, S and G have not been initialised - please run NMTF.initialise() first."        
        
        self.all_times = [] # to plot performance against time
        self.all_performances = {} # for plotting convergence of metrics
        for metric in self.metrics:
            self.all_performances[metric] = []
            
        time_start = time.time()
        for it in range(1,iterations+1):
            # Doing S first gives more interpretable results (F,G ~= [0,1] rather than [0,20])
            for k,l in itertools.product(range(0,self.K),range(0,self.L)):
                self.update_S(k,l)
                    
            for k in range(0,self.K):
                self.update_F(k)
                
            for l in range(0,self.L):
                self.update_G(l)
               
            self.give_update(it)
            
            time_iteration = time.time()
            self.all_times.append(time_iteration-time_start)  
        
        
    """ Method for doing both initialise() and run() """
    def train(self,iterations,init_S='random',init_FG='random',expo_prior=1.):
        self.initialise(init_S=init_S,init_FG=init_FG,expo_prior=expo_prior) 
        self.run(iterations=iterations)
        
                
    """ Updates for F, G, S. """                
    # Compute the dot product of three matrices
    def triple_dot(self,M1,M2,M3):
        return numpy.dot(M1,numpy.dot(M2,M3))
        
    def update_F(self,k):
        R_pred = self.triple_dot(self.F,self.S,self.G.T)
        SG = numpy.dot(self.S[k],self.G.T)
        numerator = (self.M * self.R / R_pred * SG).sum(axis=1)
        denominator = (self.M * SG).sum(axis=1)
        self.F[:,k] = self.F[:,k] * numerator / denominator
        
    def update_G(self,l):
        R_pred = self.triple_dot(self.F,self.S,self.G.T)
        FS = numpy.dot(self.F,self.S[:,l])
        numerator = ((self.M * self.R / R_pred).T * FS).T.sum(axis=0)
        denominator = (self.M.T * FS).T.sum(axis=0)
        self.G[:,l] = self.G[:,l] * numerator / denominator
        
    def update_S(self,k,l):
        R_pred = self.triple_dot(self.F,self.S,self.G.T)
        F_times_G = self.M * numpy.outer(self.F[:,k], self.G[:,l])   
        numerator = (self.R * F_times_G / R_pred).sum()
        denominator = F_times_G.sum()
        self.S[k,l] = self.S[k,l] * numerator / denominator
           
           
    ''' Functions for computing MSE, R^2 (coefficient of determination), Rp (Pearson correlation) '''
    def predict(self,M_pred):
        R_pred = self.triple_dot(self.F,self.S,self.G.T)
        MSE = self.compute_MSE(M_pred,self.R,R_pred)
        R2 = self.compute_R2(M_pred,self.R,R_pred)    
        Rp = self.compute_Rp(M_pred,self.R,R_pred)        
        return {'MSE':MSE,'R^2':R2,'Rp':Rp}        
        
    def compute_MSE(self,M,R,R_pred):
        return (M * (R-R_pred)**2).sum() / float(M.sum())
        
    def compute_R2(self,M,R,R_pred):
        mean = (M*R).sum() / float(M.sum())
        SS_total = float((M*(R-mean)**2).sum())
        SS_res = float((M*(R-R_pred)**2).sum())
        return 1. - SS_res / SS_total if SS_total != 0. else numpy.inf
        
    def compute_Rp(self,M,R,R_pred):
        mean_real = (M*R).sum() / float(M.sum())
        mean_pred = (M*R_pred).sum() / float(M.sum())
        covariance = (M*(R-mean_real)*(R_pred-mean_pred)).sum()
        variance_real = (M*(R-mean_real)**2).sum()
        variance_pred = (M*(R_pred-mean_pred)**2).sum()
        return covariance / float(math.sqrt(variance_real)*math.sqrt(variance_pred))   
        
    def compute_I_div(self):    
        R_pred = self.triple_dot(self.F,self.S,self.G.T)
        return (self.M * ( self.R_excl_unknown * numpy.log( self.R_excl_unknown / R_pred ) - self.R_excl_unknown + R_pred ) ).sum()        
        
        
    """ Give updates and store performances """
    def give_update(self,iteration):    
        perf = self.predict(self.M)
        i_div = self.compute_I_div()
        
        for metric in self.metrics:
            self.all_performances[metric].append(perf[metric])
               
        print("Iteration %s. I-divergence: %s. MSE: %s. R^2: %s. Rp: %s." % (iteration,i_div,perf['MSE'],perf['R^2'],perf['Rp']))

In [ ]:
# defining k and l for codebook k = m/5, l = n/5 ; X_src:m*n
k, l = rs_m.shape
k = int(k/5)
l = int(l/5)

In [ ]:
n_s = NMTF(rs_n, rs_m, 3, 5)

In [ ]:
n_s.initialise()

In [ ]:
n_s.run(5)

Iteration 1. I-divergence: 378.1644634140963. MSE: 0.2936201968398527. R^2: 0.7099947124054745. Rp: 0.8594888884034032.
Iteration 2. I-divergence: 52.76364457187385. MSE: 0.043608851739008817. R^2: 0.9569280392617674. Rp: 0.9782402298236144.
Iteration 3. I-divergence: 31.36383111975126. MSE: 0.025817394543029733. R^2: 0.9745004566784745. Rp: 0.9872123687231062.
Iteration 4. I-divergence: 28.30652303622776. MSE: 0.02323430932965065. R^2: 0.9770517402013711. Rp: 0.9885555609313417.
Iteration 5. I-divergence: 26.733543772262212. MSE: 0.021883689387267254. R^2: 0.9783857319670514. Rp: 0.9892379497360584.


In [ ]:
# F:U_src  S:S  G:V_src
print(n_s.F)
print(n_s.S)
print(n_s.G.T)

[[0.52737792 0.1712072  1.4031135 ]
 [0.33557281 1.53635449 0.08769523]
 [0.44208952 1.16676961 0.43305718]
 ...
 [0.87418334 0.85285377 0.8317625 ]
 [2.54768466 0.18382287 0.88061347]
 [0.78843511 0.4684465  0.32752716]]
[[1.44057556 1.22093261 1.28500614 0.7335143  0.88868203]
 [1.44837338 0.6476069  0.43070051 1.6294346  0.55115915]
 [0.43465388 1.47058078 1.32110636 1.06965559 0.4179564 ]]
[[8.65060995e-01 6.38902557e-01 4.70778028e-01 2.36644528e-01
  1.76354041e-02 4.43345284e-01 2.18848286e-02 4.24378937e-01
  4.42312793e-01 7.56160859e-01 9.71446601e-01 1.07215501e-01
  3.61264894e-01 4.09075997e-01 8.76500957e-01 3.90021801e-01
  9.00018379e-01 6.92098163e-01 4.73279470e-01 7.54638620e-01
  7.97139773e-01 3.59038925e-01 6.60247071e-01 2.35337959e-01
  6.35868990e-03 5.65372231e-01 5.80231636e-01 4.55629311e-01
  3.14787520e-01 2.07333511e-01 6.53579535e-01 8.16524474e-01
  5.58755208e-01 6.04245724e-01 2.73424479e-01 5.77265088e-01
  2.43897575e-01 5.22388553e-01 6.44447816e-0

In [ ]:
# U_src = G
U_src = n_s.F
V_src = n_s.G
print(U_src)
print(V_src)

[[0.52737792 0.1712072  1.4031135 ]
 [0.33557281 1.53635449 0.08769523]
 [0.44208952 1.16676961 0.43305718]
 ...
 [0.87418334 0.85285377 0.8317625 ]
 [2.54768466 0.18382287 0.88061347]
 [0.78843511 0.4684465  0.32752716]]
[[8.65060995e-01 6.14720052e-01 7.35509229e-01 9.00987484e-01
  7.66674696e-01]
 [6.38902557e-01 7.05932442e-01 1.92778117e-02 5.46388667e-01
  1.08252746e-01]
 [4.70778028e-01 5.42808455e-01 4.17582322e-01 5.69426278e-01
  3.85217610e-01]
 [2.36644528e-01 3.66405963e-01 3.36054418e-01 2.55006548e-01
  1.07530604e+00]
 [1.76354041e-02 8.74880906e-01 5.70603601e-01 8.40795295e-02
  9.95570455e-01]
 [4.43345284e-01 7.58098262e-01 6.81965067e-02 5.37756904e-01
  6.13062376e-01]
 [2.18848286e-02 6.52315804e-01 2.28738283e-01 1.86842491e-01
  9.33064439e-01]
 [4.24378937e-01 9.22456716e-01 6.07836058e-01 3.50977779e-01
  6.62138997e-01]
 [4.42312793e-01 8.58440994e-01 8.81689334e-01 7.68075723e-01
  5.36483075e-01]
 [7.56160859e-01 4.81178238e-01 6.05901311e-01 1.75573443e

In [ ]:
# select the most impoartant latent rating factor and mark the index = 1 else 0
K = U_src
for i in range(len(U_src)):
  m = U_src[i].max()
  for j in range(len(U_src[i])):
    if U_src[i][j] == m:
      K[i][j] = 1
    else:
      K[i][j] = 0
print(K)

L = V_src
for i in range(len(V_src)):
  m = V_src[i].max()
  for j in range(len(V_src[i])):
    if V_src[i][j] == m:
      L[i][j] = 1
    else:
      L[i][j] = 0
print(L)

[[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
[[0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0.

In [ ]:
#we add all rows of K into an identify where the value of xj records the number of users who have j as their most important latent rating factor as their most important rating latent rating factor
x = K.sum(axis=0)
print(x)
if 0 in x:
 i = x.index(0)
 numpy.delete(U_src, i, axis=0)
y = L.sum(axis=0)
print(y)
if 0 in y:
  i = y.index(0)
  numpy.delete(U_src, i, axis=0)

[3754. 3001. 2947.]
[14. 13. 14. 11. 15.]


In [ ]:
# delete the rows containing 0
if 0 in x:
 i = x.index(0)
 numpy.delete(x, i, axis=0)

if 0 in y:
  i = y.index(0)
  numpy.delete(L, i, axis=0)

In [ ]:
# combining all above and running in a loop
k, l = rs_m.shape
k = int(k/5)
l = int(l/5)
x = [0] * k
y = [0] * l
while (0 in x) and (0 in y):
  n_s = NMTF(rs_n, rs_m, k, l)
  x = [0] * k
  y = [0] * l
  n_s.initialise()
  n_s.run(1)
  U_src = n_s.G
  K = U_src
  for i in range(len(U_src)):
    m = U_src[i].max()
    for j in range(len(U_src[i])):
      if U_src[i][j] == m:
        K[i][j] = 1
      else:
        K[i][j] = 0
  x = K.sum(axis=0)
  if 0 in x:
    i = x.index(0)
    k = k - 1

  V_src = n_s.F
  L = V_src
  for i in range(len(V_src)):
    m = V_src[i].max()
    for j in range(len(V_src[i])):
      if V_src[i][j] == m:
        L[i][j] = 1
      else:
        L[i][j] = 0
  y = L.sum(axis=0)
  if 0 in y:
    i = y.index(0)
    l = l - 1

In [ ]:
# Codebook construction Eq 2 of Ref 13
import numpy as np
x = np.matmul(n_s.F.T, np.array(rs_n))
x = np.matmul(x, n_s.G)
print(x)
y = np.matmul(n_s.F.T, np.ones((n_s.F.shape[0], n_s.G.shape[0])))
y = np.matmul(y, n_s.G)
print(y)

[[1740.25       1668.5        5206.         8616.16666667 1002.        ]
 [1337.         1398.         3540.         6665.33333333  921.        ]
 [1174.         1250.         3110.         6252.          828.        ]]
[[52556. 48802. 52556. 41294. 56310.]
 [42014. 39013. 42014. 33011. 45015.]
 [41258. 38311. 41258. 32417. 44205.]]


In [ ]:
# Entry wise division to get codebook b eq 2 of ref 13
B = np.divide(x,y)
print(B.shape)
print(B)

(3, 5)
[[0.0331123  0.03418917 0.09905624 0.2086542  0.01779435]
 [0.03182273 0.03583421 0.08425763 0.20191249 0.02045985]
 [0.02845509 0.0326277  0.07537932 0.19286177 0.01873091]]


In [ ]:
n_t = NMTF(rt_n, rt_m, 3, 5)

In [ ]:
n_t.initialise()

In [ ]:
n_t.run(5)

Iteration 1. I-divergence: 369.0796951567275. MSE: 0.28226521092276763. R^2: 0.7561739826151815. Rp: 0.8774369782173064.
Iteration 2. I-divergence: 49.31125010025026. MSE: 0.03810025141176332. R^2: 0.9670882836297088. Rp: 0.9835133751269703.
Iteration 3. I-divergence: 27.942563532914182. MSE: 0.020500527701408523. R^2: 0.9822912571925512. Rp: 0.9911061673405898.
Iteration 4. I-divergence: 25.95630347018084. MSE: 0.018842765396995885. R^2: 0.9837232635639145. Rp: 0.9918428799368886.
Iteration 5. I-divergence: 25.394459716807013. MSE: 0.01837738096969656. R^2: 0.9841252714170621. Rp: 0.9920549654060264.


In [ ]:
print(n_t.F)
print(n_t.S)
print(n_t.G.T)

[[0.66788767 0.69121113 0.54217553]
 [1.08258425 0.27810021 0.52280603]
 [0.5694103  0.34593947 0.61734751]
 ...
 [1.01273007 0.61148039 0.44015048]
 [0.60535274 1.16796662 0.47997261]
 [0.44554376 0.23474533 1.37271903]]
[[1.54470584 1.41994871 1.07554513 0.65110156 0.88045941]
 [1.0285665  1.08807601 0.05764786 1.12955714 0.91313357]
 [0.97605911 0.75039372 0.04702307 0.98870717 0.86938469]]
[[0.38480571 0.50651812 0.92381235 0.85457461 0.68852688 0.08282718
  0.63369958 0.2227004  0.35076206 0.78787804 0.79115207 0.38679147
  0.54867504 0.41964228 1.1172856  0.65567363 0.81459089 0.71788168
  0.38880255 0.73791516 0.88598693 1.05779596 0.30281366 1.01900766
  0.0477364  0.41279327 0.51849815 0.20944487 0.96570029 0.86154856
  0.61917532 0.12477378 0.64621418 0.19429232 0.71269298 0.96111983
  0.65932384 0.49160389 1.10687752 0.36733807 0.09157609 0.97504017
  0.14693803 0.93302253 0.65333751 0.69807591 0.76635918 0.79597053
  0.06649123 0.96058251 0.21046378 0.50691462 0.83979062 0.

In [ ]:
U_tgt = n_t.F
V_tgt = n_t.G

In [ ]:
# reconstruction of target matrix
W = np.array(rt_m)
X_tgt = np.array(rt_n)
W_i = np.array(np.ones(W.shape) - W)

In [ ]:
rs_n

itemId,000047715X,000105001X,000171287X,000211495X,000216132X,1046519,1047566,1052292,1061240,13765,1381733,1384198,1472933,1473123,1477889,1711296,1712764,1712799,1712845,1713205,1713353,1714384,1714538,1714716,1716069,1720279,1720392,1839225,1839233,1840266,1840851,1841572,1844423,1857290,1921517,1932349,1937987,1945424,1950061,1954563,1983008,2000318,2005018,2005263,2005379,2005395,2006448,2006995,2007649,2007908,2007916,2008343,2008572,2008599,2051850,2103826,2111306,2112256,2117010,2117088,2152525,2153211,2153971,2160595,2163578,230022,913154
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A0334855HN6E38CXWXZR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0520969JGW2CZRMEZMM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0790722OCX87RKL2J3T,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A100ATCZ8R81KG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A101M64MW6EH3O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZVK41090G5LI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZVVRNCE2F0ZY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZWV6VDHSTION,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# select the most impoartant latent rating factor and mark the index = 1 else 0
Kt = U_tgt
for i in range(len(U_tgt)):
  m = U_tgt[i].max()
  for j in range(len(U_tgt[i])):
    if U_tgt[i][j] == m:
      Kt[i][j] = 1
    else:
      Kt[i][j] = 0
print(Kt)

Lt = V_tgt
for i in range(len(V_tgt)):
  m = V_tgt[i].max()
  for j in range(len(V_tgt[i])):
    if V_tgt[i][j] == m:
      Lt[i][j] = 1
    else:
      Lt[i][j] = 0
print(Lt)

In [ ]:
# equation 6 
a = np.matmul(U_tgt, B)
a = np.matmul(a, V_tgt.T)
a = 0.95 * a   # pearson coefficient = 0.95 taken manually  // pearson = np.corrcoef(np.array(rs_n),np.array(rt_n))
a = np.multiply(W_i, a)
b = np.multiply(W, X_tgt)
c = np.add(a, b)
X_tgt_b = c

In [ ]:
X_tgt_b

array([[0.08004475, 0.19181687, 0.03023159, ..., 0.19181687, 0.0340425 ,
        0.01943685],
       [0.09410343, 0.19822149, 0.03145668, ..., 0.19822149, 0.03247971,
        0.01690464],
       [0.07161035, 0.18321868, 0.02703233, ..., 0.18321868, 0.03099632,
        0.01779437],
       ...,
       [0.09410343, 0.19822149, 0.03145668, ..., 0.19822149, 0.03247971,
        0.01690464],
       [0.08004475, 0.19181687, 0.03023159, ..., 0.19181687, 0.0340425 ,
        0.01943685],
       [0.07161035, 0.18321868, 0.02703233, ..., 0.18321868, 0.03099632,
        0.01779437]])

In [ ]:
print(X_tgt_b.shape)
print(rt.shape)

(9788, 70)
(9788, 70)


## Stochastic Gradient Descent

In [ ]:
import numpy as np

def sgd(
    gradient, x, y, n_vars=None, start=None, learn_rate=0.1,
    decay_rate=0.0, batch_size=1, n_iter=50, tolerance=1e-06,
    dtype="float64", random_state=None
):
    # Checking if the gradient is callable
    if not callable(gradient):
        raise TypeError("'gradient' must be callable")

    # Setting up the data type for NumPy arrays
    dtype_ = np.dtype(dtype)

    # Converting x and y to NumPy arrays
    x, y = np.array(x, dtype=dtype_), np.array(y, dtype=dtype_)
    n_obs = x.shape[0]
    if n_obs != y.shape[0]:
        raise ValueError("'x' and 'y' lengths do not match")
    xy = np.c_[x.reshape(n_obs, -1), y.reshape(n_obs, 1)]

    # Initializing the random number generator
    seed = None if random_state is None else int(random_state)
    rng = np.random.default_rng(seed=seed)

    # Initializing the values of the variables
    vector = (
        rng.normal(size=int(n_vars)).astype(dtype_)
        if start is None else
        np.array(start, dtype=dtype_)
    )

    # Setting up and checking the learning rate
    learn_rate = np.array(learn_rate, dtype=dtype_)
    if np.any(learn_rate <= 0):
        raise ValueError("'learn_rate' must be greater than zero")

    # Setting up and checking the decay rate
    decay_rate = np.array(decay_rate, dtype=dtype_)
    if np.any(decay_rate < 0) or np.any(decay_rate > 1):
        raise ValueError("'decay_rate' must be between zero and one")

    # Setting up and checking the size of minibatches
    batch_size = int(batch_size)
    if not 0 < batch_size <= n_obs:
        raise ValueError(
            "'batch_size' must be greater than zero and less than "
            "or equal to the number of observations"
        )

    # Setting up and checking the maximal number of iterations
    n_iter = int(n_iter)
    if n_iter <= 0:
        raise ValueError("'n_iter' must be greater than zero")

    # Setting up and checking the tolerance
    tolerance = np.array(tolerance, dtype=dtype_)
    if np.any(tolerance <= 0):
        raise ValueError("'tolerance' must be greater than zero")

    # Setting the difference to zero for the first iteration
    diff = 0

    # Performing the gradient descent loop
    for _ in range(n_iter):
        # Shuffle x and y
        rng.shuffle(xy)

        # Performing minibatch moves
        for start in range(0, n_obs, batch_size):
            stop = start + batch_size
            x_batch, y_batch = xy[start:stop, :-1], xy[start:stop, -1:]

            # Recalculating the difference
            grad = np.array(gradient(x_batch, y_batch, vector), dtype_)
            diff = decay_rate * diff - learn_rate * grad

            # Checking if the absolute difference is small enough
            if np.all(np.abs(diff) <= tolerance):
                break

            # Updating the values of the variables
            vector += diff

    return vector if vector.shape else vector.item()

In [ ]:
wdwddw = np.array(rt_n)
wdwddw[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       5., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [ ]:
def ssr_gradient(x, y, b):
    res = b[0] + b[1] * x - y
    return res.mean(), (res * x).mean()

In [ ]:
x = np.array(X_tgt_b)
y = np.array(rt_n)
for i in range(len(X_tgt_b)):
  X = x[i]
  Y = y[i]
  sgd(ssr_gradient, X, Y, n_vars=2, learn_rate=0.0001, decay_rate=0.8, batch_size=3, n_iter=100_000, random_state=0)

KeyboardInterrupt: ignored

In [ ]:
X = np.array([[2,3,4,4,5],
              [3,4,5,1,3]])
Y = np.array([[2,4,2,4,6],
             [2,4,5,3,1]])

In [ ]:
n = []
for i in range(2):
  x = X[i]
  y = Y[i]
  n.append(sgd(ssr_gradient, x, y, n_vars=2, learn_rate=0.0001, decay_rate=0.8, batch_size=3, n_iter=100_000, random_state=0))
  print(n[i])

[-0.00192403  0.99826132]
[1.1782131  0.57091459]


In [ ]:
ssr_gradient()

TypeError: ignored

In [ ]:
X[i]

0.08004474698909887